# Install Torch in Google Collab
As I am switching my laptop to MacBook Air M1, and a number of compatibility issues are making experimenting on local machine quite difficult. Especially like installing packages like numpy, and when using docker container to do the assignment, cannot utilize GPU and the performance is quite low.

Hence, move the whole workspace onto Google Collab for better performance and support.

In [1]:
! pip3 install torch

# Import all required packages

In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
%load_ext autoreload
%autoreload 2

import torch
import numpy as np
import pandas as pd
from joblib import dump, load
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from src.data.sets import split_sets_by_time, save_sets, split_sets_random

## Load data from S3 bucket
As uploading onto Google Collab caused incomplete loading of the data, loading the data from S3 turned out to be reliable and fast enough.

In [3]:
df = pd.read_csv('https://uts-advdsi.s3-ap-southeast-2.amazonaws.com/beer_reviews.csv')

## Data Exploration

In [4]:
df.describe()

,brewery_id,review_time,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv,beer_beerid
count,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.586614e+06,1.518829e+06,1.586614e+06
mean,3.130099e+03,1.224089e+09,3.815581e+00,3.735636e+00,3.841642e+00,3.743701e+00,3.792860e+00,7.042387e+00,2.171279e+04
std,5.578104e+03,7.654427e+07,7.206219e-01,6.976167e-01,6.160928e-01,6.822184e-01,7.319696e-01,2.322526e+00,2.181834e+04
min,1.000000e+00,8.406720e+08,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e-02,3.000000e+00
25%,1.430000e+02,1.173224e+09,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,3.500000e+00,5.200000e+00,1.717000e+03
50%,4.290000e+02,1.239203e+09,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,6.500000e+00,1.390600e+04
75%,2.372000e+03,1.288568e+09,4.500000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.500000e+00,8.500000e+00,3.944100e+04
max,2.800300e+04,1.326285e+09,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.770000e+01,7.731700e+04


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   brewery_id          1586614 non-null  int64  
 1   brewery_name        1586599 non-null  object 
 2   review_time         1586614 non-null  int64  
 3   review_overall      1586614 non-null  float64
 4   review_aroma        1586614 non-null  float64
 5   review_appearance   1586614 non-null  float64
 6   review_profilename  1586266 non-null  object 
 7   beer_style          1586614 non-null  object 
 8   review_palate       1586614 non-null  float64
 9   review_taste        1586614 non-null  float64
 10  beer_name           1586614 non-null  object 
 11  beer_abv            1518829 non-null  float64
 12  beer_beerid         1586614 non-null  int64  
dtypes: float64(6), int64(3), object(4)
memory usage: 157.4+ MB


We can see some missing data, and there are features won't be available in Production as inputs. We will be replacing missing value with special value, and dropping features that will not be available in Production.

## Data Cleanup

Dropping features that will not be available in Production, and fill in missing value (brewery_name) as 'OTHERS'

In [6]:
df_cleaned = df.copy()

In [7]:
df_cleaned = df_cleaned.drop(['brewery_id','review_time','review_overall','review_profilename','beer_name','beer_abv','beer_beerid'], axis=1)

In [8]:
df_cleaned = df_cleaned.fillna(value = 'OTHER')

## Feature Engineering
brewery_name has 5000+ unique value, and it basically crashes Google Collab when it's done through one hot encoding and adding 5000+ columns. However, using ordinal encoding doesn't quite make sense in this case. Therefore, try to tackle this issue from different angles.

When we think about what types of beer a brewery can produce, the possiblity will be a collection of beers in the total beer types (104 types). Instead of treating brewery as a name/label, we are looking at it like what types of beer this brewery can produce, and use that to identify and limit the options for better prediction performance.

In [9]:
sc = StandardScaler()    #ANN needs to standardize all features
ore = OrdinalEncoder()   #ANN Classification needs the target value to be 0 ~ N-1
ohe = OneHotEncoder()    #Use one hot encoding for better performance

Convert target value to be between 0~N-1

In [10]:
df_cleaned['beer_style'] = ore.fit_transform(df_cleaned[['beer_style']])
df_cleaned['beer_style'] = df_cleaned['beer_style'].astype(int)
df_cleaned.head()

,brewery_name,review_aroma,review_appearance,beer_style,review_palate,review_taste
0,Vecchio Birraio,2.0,2.5,65,1.5,1.5
1,Vecchio Birraio,2.5,3.0,51,3.0,3.0
2,Vecchio Birraio,2.5,3.0,59,3.0,3.0
3,Vecchio Birraio,3.0,3.5,61,2.5,3.0
4,Caldera Brewing Company,4.5,4.0,9,4.0,4.5


For each observation, we put beer_style into one hot encoding, so later we can group based on brewery to engineering new features that what types of beer each brewery can produce.

In [11]:
X_brewer = pd.DataFrame(ohe.fit_transform(df_cleaned[['beer_style']]).toarray())

In [12]:
df_cat = pd.concat([df_cleaned['brewery_name'], X_brewer], axis=1)
df_cat.head()

,brewery_name,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103
0,Vecchio Birraio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Vecchio Birraio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Vecchio Birraio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Vecchio Birraio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Caldera Brewing Company,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Group by brewery, and select max value of each column, which translate to a tick (Ture) if a brewery is able to produce that type of beer.

In [13]:
df_cat = df_cat.groupby('brewery_name').max()
df_cat.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103
brewery_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
't Hofbrouwerijke,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
(512) Brewing Company,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
10 Barrel Brewing Co.,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1516 Brewing Company,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16 Mile Brewing Company,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Save df_cat as a mapping table for Production prediction.

In [14]:
dump(df_cat,'df_cat')
dump(ore,'OrdinalEncoderTarget')
dump(ohe,'OneHotEncoderBeerType')

['OneHotEncoderBeerType']

Combine the new features into the dataset by joining the brewery_name column.

In [15]:
df_convert = df_cleaned.join(df_cat,on='brewery_name')
df_convert = df_convert.drop(columns=['brewery_name'])
df_convert.head()

,review_aroma,review_appearance,beer_style,review_palate,review_taste,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,...,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103
0,2.0,2.5,65,1.5,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.5,3.0,51,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.5,3.0,59,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,3.5,61,2.5,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.5,4.0,9,4.0,4.5,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Standardization

In [16]:
num_cols = ['review_aroma', 'review_appearance', 'review_palate', 'review_taste']
df_convert[num_cols] = sc.fit_transform(df_convert[num_cols])
dump(sc,'standardScaler')

['standardScaler']

## Split data and save them

In [17]:
X_train, y_train, X_val, y_val, X_test, y_test = split_sets_random(df_convert, target_col='beer_style', test_ratio=0.2)

In [18]:
save_sets(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, X_test=X_test, y_test=y_test, path='/')

# Get Baseline Performance

In [21]:
from src.models.null import NullModel

In [22]:
baseline_model = NullModel(target_type='classification')
y_base = baseline_model.fit_predict(y_train)

In [23]:
from src.models.performance import print_class_perf

In [24]:
print_class_perf(y_base, y_train, set_name='Training', average='weighted')

Accuracy Training: 0.0742157299405022
F1 Training: 0.01025487603110527


The accuracy is pretty low in the baseline model 7.4%

# Model Selection - Neural Network Setup

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from src.models.pytorch import PytorchDataset, get_device

In [20]:
train_dataset = PytorchDataset(X=X_train, y=y_train)
val_dataset = PytorchDataset(X=X_val, y=y_val)
test_dataset = PytorchDataset(X=X_test, y=y_test)

In [25]:
class PytorchMultiClass(nn.Module):
    def __init__(self, num_features):
        super(PytorchMultiClass, self).__init__()
        
        self.layer_1 = nn.Linear(num_features, 128)
        self.layer_out = nn.Linear(128, 104)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = F.dropout(F.relu(self.layer_1(x)), training=self.training)
        x = self.layer_out(x)
        return self.softmax(x)

Utilize GPU on Google Collab

In [27]:
device = get_device()
device

device(type='cuda', index=0)

In [28]:
model = PytorchMultiClass(X_train.shape[1])
model.to(device)

PytorchMultiClass(
  (layer_1): Linear(in_features=108, out_features=128, bias=True)
  (layer_out): Linear(in_features=128, out_features=104, bias=True)
  (softmax): Softmax(dim=1)
)

In [29]:
criterion = nn.CrossEntropyLoss()

## Train the model - forward function

In [30]:
def train_classification(train_data, model, criterion, optimizer, batch_size, device, scheduler=None, generate_batch=None):
    """Train a Pytorch multi-class classification model

    Parameters
    ----------
    train_data : torch.utils.data.Dataset
        Pytorch dataset
    model: torch.nn.Module
        Pytorch Model
    criterion: function
        Loss function
    optimizer: torch.optim
        Optimizer
    bacth_size : int
        Number of observations per batch
    device : str
        Name of the device used for the model
    scheduler : torch.optim.lr_scheduler
        Pytorch Scheduler used for updating learning rate
    collate_fn : function
        Function defining required pre-processing steps

    Returns
    -------
    Float
        Loss score
    Float:
        Accuracy Score
    """
    
    # Set model to training mode
    model.train()
    train_loss = 0
    train_acc = 0
    
    # Create data loader
    data = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=generate_batch)
    
    # Iterate through data by batch of observations
    for feature, target_class in data:

        # Reset gradients
        optimizer.zero_grad()
        
        # Load data to specified device
        feature, target_class = feature.to(device), target_class.to(device)
        
        # Make predictions
        output = model(feature)
        
        # Calculate loss for given batch
        loss = criterion(output, target_class.long())

        # Calculate global loss
        train_loss += loss.item()
        
        # Calculate gradients
        loss.backward()

        # Update Weights
        optimizer.step()
        
        # Calculate global accuracy
        train_acc += (output.argmax(1) == target_class).sum().item()

    # Adjust the learning rate
    if scheduler:
        scheduler.step()

    train_loss = train_loss / len(train_data)
    train_acc = train_acc / len(train_data)

    return train_loss, train_acc

In [31]:
def test_classification(test_data, model, criterion, batch_size, device, generate_batch=None):
    """Calculate performance of a Pytorch multi-class classification model

    Parameters
    ----------
    test_data : torch.utils.data.Dataset
        Pytorch dataset
    model: torch.nn.Module
        Pytorch Model
    criterion: function
        Loss function
    bacth_size : int
        Number of observations per batch
    device : str
        Name of the device used for the model
    collate_fn : function
        Function defining required pre-processing steps

    Returns
    -------
    Float
        Loss score
    Float:
        Accuracy Score
    """    
    
    # Set model to evaluation mode
    model.eval()
    test_loss = 0
    test_acc = 0
    
    # Create data loader
    data = DataLoader(test_data, batch_size=batch_size, collate_fn=generate_batch)
    
    # Iterate through data by batch of observations
    for feature, target_class in data:
        
        # Load data to specified device
        feature, target_class = feature.to(device), target_class.to(device)
        
        # Set no update to gradients
        with torch.no_grad():
            
            # Make predictions
            output = model(feature)
            
            # Calculate loss for given batch
            loss = criterion(output, target_class.long())

            # Calculate global loss
            test_loss += loss.item()
            
            # Calculate global accuracy
            test_acc += (output.argmax(1) == target_class).sum().item()

    return test_loss / len(test_data), test_acc / len(test_data)

## Round 1 Training - Higher learning rate

In [32]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
N_EPOCHS = 5000
BATCH_SIZE = 1000

In [33]:
for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_classification(train_dataset, model=model, criterion=criterion, optimizer=optimizer, batch_size=BATCH_SIZE, device=device)
    valid_loss, valid_acc = test_classification(val_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)

    print(f'Epoch: {epoch}')
    print(f'\t(train)\t|\tLoss: {train_loss:.4f}\t|\tAcc: {train_acc * 100:.1f}%')
    print(f'\t(valid)\t|\tLoss: {valid_loss:.4f}\t|\tAcc: {valid_acc * 100:.1f}%')

Epoch: 0
	(train)	|	Loss: 0.0045	|	Acc: 17.8%
	(valid)	|	Loss: 0.0045	|	Acc: 19.9%
Epoch: 1
	(train)	|	Loss: 0.0045	|	Acc: 19.9%
	(valid)	|	Loss: 0.0045	|	Acc: 20.5%
Epoch: 2
	(train)	|	Loss: 0.0045	|	Acc: 20.3%
	(valid)	|	Loss: 0.0045	|	Acc: 21.1%
Epoch: 3
	(train)	|	Loss: 0.0045	|	Acc: 21.0%
	(valid)	|	Loss: 0.0045	|	Acc: 21.5%
Epoch: 4
	(train)	|	Loss: 0.0044	|	Acc: 21.4%
	(valid)	|	Loss: 0.0045	|	Acc: 22.0%
Epoch: 5
	(train)	|	Loss: 0.0044	|	Acc: 21.8%
	(valid)	|	Loss: 0.0044	|	Acc: 22.5%
Epoch: 6
	(train)	|	Loss: 0.0044	|	Acc: 22.0%
	(valid)	|	Loss: 0.0044	|	Acc: 22.5%
Epoch: 7
	(train)	|	Loss: 0.0044	|	Acc: 22.1%
	(valid)	|	Loss: 0.0044	|	Acc: 22.7%
Epoch: 8
	(train)	|	Loss: 0.0044	|	Acc: 22.3%
	(valid)	|	Loss: 0.0044	|	Acc: 22.7%
Epoch: 9
	(train)	|	Loss: 0.0044	|	Acc: 22.3%
	(valid)	|	Loss: 0.0044	|	Acc: 22.9%
Epoch: 10
	(train)	|	Loss: 0.0044	|	Acc: 22.4%
	(valid)	|	Loss: 0.0044	|	Acc: 22.9%
Epoch: 11
	(train)	|	Loss: 0.0044	|	Acc: 22.4%
	(valid)	|	Loss: 0.0044	|	Acc: 22.9%
Ep

KeyboardInterrupt: ignored

In [34]:
torch.save(model.state_dict(), 'torch24.model.state_dict')

In here, we already see much better accuracy compared to the baseline model. However, we do see the model is hitting it's performance ceiling. Try reduce the learning rate to tune the performance further.

## Round 2 Training - lower learning rate

In [35]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [36]:
for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_classification(train_dataset, model=model, criterion=criterion, optimizer=optimizer, batch_size=BATCH_SIZE, device=device)
    valid_loss, valid_acc = test_classification(val_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)

    print(f'Epoch: {epoch}')
    print(f'\t(train)\t|\tLoss: {train_loss:.4f}\t|\tAcc: {train_acc * 100:.1f}%')
    print(f'\t(valid)\t|\tLoss: {valid_loss:.4f}\t|\tAcc: {valid_acc * 100:.1f}%')

Epoch: 0
	(train)	|	Loss: 0.0044	|	Acc: 24.2%
	(valid)	|	Loss: 0.0044	|	Acc: 24.7%
Epoch: 1
	(train)	|	Loss: 0.0044	|	Acc: 24.3%
	(valid)	|	Loss: 0.0044	|	Acc: 24.7%
Epoch: 2
	(train)	|	Loss: 0.0044	|	Acc: 24.3%
	(valid)	|	Loss: 0.0044	|	Acc: 24.7%
Epoch: 3
	(train)	|	Loss: 0.0044	|	Acc: 24.3%
	(valid)	|	Loss: 0.0044	|	Acc: 24.7%
Epoch: 4
	(train)	|	Loss: 0.0044	|	Acc: 24.4%
	(valid)	|	Loss: 0.0044	|	Acc: 24.7%
Epoch: 5
	(train)	|	Loss: 0.0044	|	Acc: 24.4%
	(valid)	|	Loss: 0.0044	|	Acc: 24.8%
Epoch: 6
	(train)	|	Loss: 0.0044	|	Acc: 24.4%
	(valid)	|	Loss: 0.0044	|	Acc: 24.8%
Epoch: 7
	(train)	|	Loss: 0.0044	|	Acc: 24.4%
	(valid)	|	Loss: 0.0044	|	Acc: 24.8%
Epoch: 8
	(train)	|	Loss: 0.0044	|	Acc: 24.5%
	(valid)	|	Loss: 0.0044	|	Acc: 24.8%
Epoch: 9
	(train)	|	Loss: 0.0044	|	Acc: 24.5%
	(valid)	|	Loss: 0.0044	|	Acc: 24.8%
Epoch: 10
	(train)	|	Loss: 0.0044	|	Acc: 24.5%
	(valid)	|	Loss: 0.0044	|	Acc: 24.8%
Epoch: 11
	(train)	|	Loss: 0.0044	|	Acc: 24.5%
	(valid)	|	Loss: 0.0044	|	Acc: 24.8%
Ep

KeyboardInterrupt: ignored

In [37]:
torch.save(model.state_dict(), 'torch25.model.state_dict')

In [39]:
valid_loss, valid_acc = test_classification(test_dataset, model=model, criterion=criterion, batch_size=5000, device=device)
print(f'\t(valid)\t|\tLoss: {valid_loss:.4f}\t|\tAcc: {valid_acc * 100:.1f}%')

	(valid)	|	Loss: 0.0009	|	Acc: 25.0%


With further tuning, the performance has gone up to 25% accuracy, and it's consistent across training, validation and testing sets.

In [63]:
print(model)

PytorchMultiClass(
  (layer_1): Linear(in_features=108, out_features=128, bias=True)
  (layer_out): Linear(in_features=128, out_features=104, bias=True)
  (softmax): Softmax(dim=1)
)
